In [ ]:
!pip install fosforml

In [1]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()

In [2]:
from fosforml.model_manager import snowflakesession
session_instance = snowflakesession()
params = session_instance.connection_params
print(params)

{'user': 'PRAKHARGOYAL', 'password': 'Prak@9818', 'account': 'fya62509.us-east-1', 'role': 'PRAKHARGOYAL', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'PRAKHAR_TEST_DB', 'schema': 'PRAKHAR_TEST_SCHEMA'}


#### Snowflake Modeling

In [3]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [4]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [7]:
# snow_df = session.table("HR_CHURN")
import pandas as pd
df =pd.read_csv("HR_CHURN.csv")
df.head()
snow_df = session.create_dataframe(df)
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])

,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,DEPARTMENT,SALARY
0,0,0.38,0.53,2,157,3,0,1,0,sales,low
1,1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,3,0.72,0.87,5,223,5,0,1,0,sales,low
4,4,0.37,0.52,2,159,3,0,1,0,sales,low


In [ ]:
input_data_frame = input_data_frame.drop("Unnamed: 0")
test_df = test_df.drop("Unnamed: 0")

In [ ]:
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [ ]:
input_data_frame.show()

In [ ]:
pipeline = GradientBoostingClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)

In [ ]:
pipeline.label_cols

In [ ]:
!pip freeze | grep -i sci

In [ ]:
test_df = apply_label_encoding(test_df)

In [ ]:
pred_df = pipeline.predict(test_df)

In [ ]:
pred_df.show()

#### Model Registration,

In [ ]:
from fosforml import register_model

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  name="SAURABH_HR_CHURN_22July",
  snowflake_df=pred_df,
  dataset_name="HR_CHURN",
  dataset_source="SnowflakeDataset",
  description="This is a test model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

In [ ]:
session.sql("SELECT TO_CHAR(CURRENT_TIMESTAMP,'YYYY-MM-DD HH12:MI:SS PM')").to_pandas().squeeze()

In [ ]:
current_data = pipeline.predict_proba(test_df)
predict_col = list(pipeline.predict(test_df).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
current = None
for i in range(len(predict_col)):
    current = current_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [ ]:
current_pandas = current.to_pandas()
current_pandas.head()

In [ ]:
reference_data = pipeline.predict_proba(input_data_frame)
predict_col = list(pipeline.predict(input_data_frame).to_pandas()['OUTPUT_SALARY'].to_numpy())
from snowflake.snowpark.functions import lit
reference = None
for i in range(len(predict_col)):
    reference = reference_data.with_column('OUTPUT_SALARY',lit(predict_col[i]))

In [ ]:
reference_pandas = reference.to_pandas()
reference_pandas.head()

In [ ]:
inference_pandas_data = test_df.to_pandas()

In [ ]:
session.write_pandas(reference_pandas,table_name="HR_REFERENCE_DATA",overwrite=True).show()

In [ ]:
session.write_pandas(current_pandas,table_name="HR_CURRENT_DATA",overwrite=True).show()

In [ ]:
session.write_pandas(inference_pandas_data,table_name="HR_INFERENCE_DATA",overwrite=True).show()